# Importar librerías

In [107]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Importar base de datos

In [108]:
df = pd.read_csv('Datasets/datos_limpios.csv')
df.job_level = df.job_level.astype(str)
df.stock_option_level = df.stock_option_level.astype(str)
df.training_times_last_year = df.training_times_last_year.astype(str)

In [109]:
df.shape

(4401, 27)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   employee_id                 4401 non-null   int64  
 1   age                         4401 non-null   int64  
 2   attrition                   4401 non-null   object 
 3   business_travel             4401 non-null   object 
 4   department                  4401 non-null   object 
 5   distance_from_home          4401 non-null   int64  
 6   education                   4401 non-null   object 
 7   education_field             4401 non-null   object 
 8   gender                      4401 non-null   object 
 9   job_level                   4401 non-null   object 
 10  job_role                    4401 non-null   object 
 11  marital_status              4401 non-null   object 
 12  monthly_income              4401 non-null   int64  
 13  num_companies_worked        4401 

# Análisis de correlación

## Correlación de variables numéricas

Seleccionar variables numéricas

In [111]:
df_numericas = df.select_dtypes(include=['int64', 'float64'])

Obtener matriz de correlaciones

In [112]:
corr_matrix = df_numericas.drop(['employee_id'], axis=1).corr()
corr_matrix = corr_matrix.apply(lambda x: np.round(x, 2))
corr_matrix

,age,distance_from_home,monthly_income,num_companies_worked,percent_salary_hike,total_working_years,years_at_company,years_since_last_promotion,years_with_curr_manager,mean_time
age,1.00,0.01,-0.04,0.30,-0.03,0.68,0.31,0.22,0.20,-0.00
distance_from_home,0.01,1.00,-0.02,-0.01,0.04,0.01,0.03,0.00,0.02,0.01
monthly_income,-0.04,-0.02,1.00,-0.02,0.00,-0.03,0.00,0.06,0.02,-0.01
num_companies_worked,0.30,-0.01,-0.02,1.00,0.03,0.24,-0.12,-0.04,-0.11,-0.02
percent_salary_hike,-0.03,0.04,0.00,0.03,1.00,-0.02,-0.03,-0.03,-0.04,0.03
total_working_years,0.68,0.01,-0.03,0.24,-0.02,1.00,0.63,0.40,0.46,-0.01
years_at_company,0.31,0.03,0.00,-0.12,-0.03,0.63,1.00,0.62,0.77,-0.02
years_since_last_promotion,0.22,0.00,0.06,-0.04,-0.03,0.40,0.62,1.00,0.51,-0.01
years_with_curr_manager,0.20,0.02,0.02,-0.11,-0.04,0.46,0.77,0.51,1.00,-0.03
mean_time,-0.00,0.01,-0.01,-0.02,0.03,-0.01,-0.02,-0.01,-0.03,1.00


Mapa de calor de correlaciones

In [120]:
fig = px.imshow(corr_matrix, text_auto=True)
fig.layout.update(height=500)
fig.show()

Las correlaciones más significativas son:

- Entre edad y total de años trabajando. Con una correlación de 0.68, tiene sentido que los empleados de más edad sean los que lleven más años trabajando.

- Entre total de años trabajando y total de años en la compañía. Los empleados que llevan más años como empleados tienden a ser los que llevan más años en la compañía.

- Entre años en la compañía y años desde la última promoción. Entre más tiempo tenga el empleado en la compañía es más probable que haya pasado mucho tiempo desde su último ascenso.

- Entre años en la compañía y años al mando del jefe actual.

Todas las variables relacionadas con el tiempo de estancia del empleado en la empresa tienen una correlación alta. 'years_at_company' y 'years_with_curr_manager' se pueden considerar variables redundantes y sería bueno eliminar una de estas, veremos qué resultados obtenemos con otros métodos de selección de variables como la regularización.

## Correlación entre la variable objetivo y variables categóricas

In [79]:
def corr_cat(df, var1, var2):
    from scipy.stats import chi2_contingency
    tabla_contingencia = pd.crosstab(df[var1], df[var2])
    display(tabla_contingencia)
    chi2, p, _, _ = chi2_contingency(tabla_contingencia)
    print('Estadístico de Chi-cuadrado:', chi2)
    print('Valor p:', p)